In [ ]:
import cv2
import numpy as np
import os

def calculate_iou(box1, box2):
    # Box format: [x_center, y_center, width, height]
    
    # Calculate the (x, y)-coordinates of the intersection rectangle
    x1 = max(box1[0] - box1[2] // 2, box2[0] - box2[2] // 2)
    y1 = max(box1[1] - box1[3] // 2, box2[1] - box2[3] // 2)
    x2 = min(box1[0] + box1[2] // 2, box2[0] + box2[2] // 2)
    y2 = min(box1[1] + box1[3] // 2, box2[1] + box2[3] // 2)

    # Compute the area of the intersection rectangle
    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Compute the area of both boxes
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]

    # Compute the area of the union
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area if union_area != 0 else 0
    return iou

def is_overlap_detected(new_box, existing_boxes, threshold=0.5):
    """
    Check if the new bounding box overlaps with any existing boxes.
    
    Parameters:
    - new_box: A tuple/list with format [x_center, y_center, width, height]
    - existing_boxes: A list of existing boxes, each in the format [x_center, y_center, width, height]
    - threshold: IoU threshold for determining overlap

    Returns:
    - True if overlap is detected, False otherwise
    """
    for existing_box in existing_boxes:
        iou = calculate_iou(new_box, existing_box)
        if iou > threshold:
            return True  # There is an overlap
    return False  # No overlap detected

# Load YOLO model
def load_yolo_model():
    # Load YOLO network
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    
    # Get the names of all layers
    layer_names = net.getLayerNames()
    
    # Get the output layers (those are the layers where YOLO outputs detections)
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    return net, output_layers


# Detect traffic lights
def detect_traffic_lights(frame, net, output_layers):
    # Get the shape of the image
    height, width, channels = frame.shape
    
    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    
    # Run forward pass to get predictions
    detections = net.forward(output_layers)
    
    # List to store detected traffic lights
    traffic_lights = []
    
    # Process the detections
    for out in detections:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # If detection confidence is high enough and only if it is traffic light

            
            if class_id==9 and confidence > 0.5:
                # Get bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                #Checkes if the new area detected have already been captured
                #Pass if true
                if is_overlap_detected((center_x, center_y, w, h), traffic_lights):
                    continue
                
                # Draw rectangle around traffic light
                cv2.rectangle(frame, (center_x - w // 2, center_y - h // 2),
                              (center_x + w // 2, center_y + h // 2), (0, 255, 0), 2)
                
                # Add label and confidence
                cv2.putText(frame, "Traffic Light", (center_x - w // 2, center_y - h // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                traffic_lights.append((center_x, center_y, w, h))
    
    return frame, traffic_lights

def main():
    # Load YOLO model
    net, output_layers = load_yolo_model()

    for image_path in os.listdir("img"):
        print("image_path",image_path)
        # Load an image
        frame = cv2.imread('img/'+image_path)
        
        if frame is None:
            print("Error: Unable to load the image.")
            return
        
        # Detect traffic lights
        frame_with_detections, traffic_lights = detect_traffic_lights(frame, net, output_layers)
        
        # Display the results
        cv2.imshow("Traffic Light Detection", frame_with_detections)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


image_path 00009.jpg
image_path 00035.jpg
image_path 1.jpg
image_path 2.jpg
image_path 3.jpg


In [6]:
#mp4 version
import cv2
import numpy as np

def calculate_iou(box1, box2):
    # Box format: [x_center, y_center, width, height]
    x1 = max(box1[0] - box1[2] // 2, box2[0] - box2[2] // 2)
    y1 = max(box1[1] - box1[3] // 2, box2[1] - box2[3] // 2)
    x2 = min(box1[0] + box1[2] // 2, box2[0] + box2[2] // 2)
    y2 = min(box1[1] + box1[3] // 2, box2[1] + box2[3] // 2)

    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area if union_area != 0 else 0
    return iou

def is_overlap_detected(new_box, existing_boxes, threshold=0.5):
    for existing_box in existing_boxes:
        iou = calculate_iou(new_box, existing_box)
        if iou > threshold:
            return True
    return False

def load_yolo_model():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

def detect_traffic_lights(frame, net, output_layers):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    traffic_lights = []
    for out in detections:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if class_id == 9 and confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                if is_overlap_detected((center_x, center_y, w, h), traffic_lights):
                    continue

                cv2.rectangle(frame, (center_x - w // 2, center_y - h // 2),
                              (center_x + w // 2, center_y + h // 2), (0, 255, 0), 2)
                cv2.putText(frame, "Traffic Light", (center_x - w // 2, center_y - h // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                traffic_lights.append((center_x, center_y, w, h))

    return frame, traffic_lights

def main():
    net, output_layers = load_yolo_model()

    # Open the video file
    video_path = "video.mp4"  # Replace with your MP4 file path
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break  # End of video

        # Detect traffic lights
        frame_with_detections, traffic_lights = detect_traffic_lights(frame, net, output_layers)

        # Display the results
        cv2.imshow("Traffic Light Detection", frame_with_detections)

        # Press 'q' to exit the loop early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
